In [1]:
import pandas as pd 
from langchain_groq.chat_models import ChatGroq

In [ ]:
import os

GROQ_API_KEY = os.getenv('GROQ_API_KEY')


groq_models = {
    "llama3-70b": "llama3-70b-8192",
    "mixtral": "mixtral-8x7b-32768",
    "gemma-7b": "gemma-7b-it",
    "llama3.1-70b": "llama-3.1-70b-versatile",
    "llama3-8b": "llama3-8b-8192",
    "llama3.1-8b": "llama-3.1-8b-instant",
    "gemma-9b": "gemma2-9b-it"
}